In [1]:
import numpy as np
import math

In [2]:
def im2col(filter_shape, 
           data, 
           stride_h=1, 
           stride_w=1,
           dilate_h=1, 
           dilate_w=1,
           pad=[0,0,0,0] # left right top bottom
          ):
    
    filter_h = (filter_shape[1]-1) * dilate_h + 1
    filter_w = (filter_shape[2]-1) * dilate_w + 1
    channels = data.shape[-1]
    
    LEFT = 0
    RIGHT = 1
    TOP = 2
    BOTTOM = 3
    

    def get_pixel(row, col):
        if 0 <= row < data.shape[1] and 0 <= col < data.shape[2]:
            return data[0, row, col, :]
        return np.zeros(channels)

    pixels_per_col = filter_shape[1] * filter_shape[2] * filter_shape[3]
    cols = np.empty((pixels_per_col, 0))
    
    # Loop over image
    for r in range(-pad[TOP], data.shape[1] - filter_h + 1 + pad[BOTTOM], stride_h):
        for c in range(-pad[LEFT], data.shape[2] - filter_w + 1 + pad[RIGHT], stride_w):
            
            # Loop over kernel
            new_col = []
            for row in range(r, r + filter_h, dilate_h):
                for col in range(c, c + filter_w, dilate_w):
                    
                    for val in get_pixel(row, col):
                        new_col.append(val)
            new_col = np.array(new_col).reshape((-1, 1))
            cols = np.append(cols, new_col, axis=-1)
                    
    return cols

In [3]:
def to_rgb(fltr, filter_shape):
    return np.stack([fltr.reshape(filter_shape) for _ in range(3)], axis=-1).reshape(-1)

In [4]:
def depthwise_conv(filters, f_shape, im2col, og_shape):
    d_mult = int(len(filters) / 3)
    channel = 0
    out = []
    new_channel_shape = (og_shape[0] - f_shape[0] + 1, og_shape[1] - f_shape[1] + 1)
    for i in range(0, len(filters)):
        out.append((filters[i] @ im2col[channel::3]).reshape(new_channel_shape))
        if i > 0 and i % d_mult == 0:
            channel += 1
    return np.stack(out, axis=-1)

In [5]:
def avg_pool(f_shape, im2col, og_shape):
    n = np.prod(f_shape) # total elements in filter
    fltr = np.array([1/n] * n)
    
    out = []
    new_channel_shape = (og_shape[0] - f_shape[0] + 1, og_shape[1] - f_shape[1] + 1)
    for channel in range(0, 3):
        out.append((fltr @ im2col[channel::3]).reshape(new_channel_shape))
    return np.stack(out, axis=-1)

In [6]:
import flatbuffers as fb
from tflite import * 

#print(dir(BuiltinOptions))

In [7]:
# load model stuf
buf = open('mobilenet_v1_1.0_224.tflite', 'rb').read()
buf = bytearray(buf)

mobilenet = Model.GetRootAsModel(buf, 0)

sub = mobilenet.Subgraphs(0)

In [8]:
 def get_padding(pad_str, filter_shape, in_shape, stride_h, stride_w, dilate_h_factor, dilate_w_factor):
    padding=[0,0,0,0]

    HEIGHT = 1
    WIDTH = 2

    if pad_str == 'SAME':
        dilated_w = (filter_shape[WIDTH] - 1) * dilate_w_factor + 1
        dilated_h = (filter_shape[HEIGHT] - 1) * dilate_h_factor + 1

        out_w = math.ceil(in_shape[WIDTH] / stride_w)
        out_h = math.ceil(in_shape[HEIGHT] / stride_h)

        padding[0] = math.floor(dilated_w / 2)
        padding[1] = out_w * stride_w - (stride_w - 1) + math.floor(filter_shape[WIDTH] / 2) - in_shape[WIDTH]
        padding[2] = math.floor(dilated_h / 2)
        padding[3] = out_h * stride_h - (stride_h - 1) + math.floor(filter_shape[HEIGHT] / 2) - in_shape[HEIGHT]                

    return padding
       
    
def get_final_shape(in_shape, filter_shape, pad_str, stride_w, stride_h):
    WIDTH = 1
    HEIGHT = 2

    w = 0
    h = 0

    if pad_str == 'SAME':
        w = math.ceil(in_shape[WIDTH] / stride_w)
        h = math.ceil(in_shape[HEIGHT] / stride_h)
    else:
        w = math.floor(in_shape[WIDTH] / stride_w)
        h = math.floor(in_shape[HEIGHT] / stride_h)

    return (h, w)


def conv2d(filter_weights, input_data, stride_h, stride_w, dilate_h, dilate_w, pad_str, bias_weights):
    padding = get_padding(pad_str, filter_shape, input_data.shape, stride_h, stride_w, dilate_h, dilate_w)

    # display stuff and make sure it's correct
    print('filter shape:', filter_weights.shape)
    print('input shape:', input_data.shape)
    print('stride_h:', stride_h)
    print('stride_w:', stride_w)
    print('dilate_h_factor:', dilate_h)
    print('dilate_w_factor:', dilate_w)
    print('padding:', pad_str, padding)

    my_im2col = im2col(filter_weights.shape, 
                       input_data, 
                       stride_h=stride_h,
                       stride_w=stride_w, 
                       dilate_h=dilate_h, 
                       dilate_w=dilate_w, 
                       pad=padding
                      )

    # matrix mult :P    
    out = filter_weights.reshape((filter_weights.shape[0], -1)) @ my_im2col
    shape = get_final_shape(input_data.shape, filter_weights.shape, pad_str, stride_w, stride_h)
    out = out.reshape((input_data.shape[0], shape[0], shape[1], filter_weights.shape[0]))

    print('output shape:', out.shape)
    return np.add(out, bias_weights)
               
    
def relu6(tensor):
    return np.min(np.max(tensor, 0), 6)

In [9]:
# dont kno what to do w this for now
#     activation_to_str = {ActivationFunctionType.NONE: 'NONE', ActivationFunctionType.RELU: 'RELU', ActivationFunctionType.RELU_N1_TO_1: 'RELU_N1_TO_1', ActivationFunctionType.RELU6: 'RELU6', ActivationFunctionType.TANH: 'TANH', ActivationFunctionType.SIGN_BIT: 'SIGN_BIT'}

layer = 0
op = sub.Operators(layer)


# get padding info from options
if op.BuiltinOptionsType() == BuiltinOptions.Conv2DOptions:
    filters_idx = op.Inputs(1)
    bias_idx = op.Inputs(2)
    filter_shape = tuple(sub.Tensors(layer).ShapeAsNumpy())
    bias_weights = np.frombuffer(mobilenet.Buffers(sub.Tensors(bias_idx).Buffer()).DataAsNumpy(), dtype=np.float32)
    filter_weights = np.frombuffer(mobilenet.Buffers(sub.Tensors(filters_idx).Buffer()).DataAsNumpy(), dtype=np.float32).reshape(filter_shape)

    union = op.BuiltinOptions()
    options = Conv2DOptions()
    options.Init(union.Bytes, union.Pos)
    
    padding_to_str = {Padding.SAME: 'SAME', Padding.VALID: 'VALID'}
    pad_str = padding_to_str[options.Padding()]

    stride_h = options.StrideH()
    stride_w = options.StrideW()
    dilate_h = options.DilationHFactor()
    dilate_w = options.DilationWFactor()

    my_input = np.zeros(224*224*3).reshape((1,224,224,3))

    out = conv2d(filter_weights, my_input, stride_h, stride_w, dilate_h, dilate_w, pad_str, bias_weights)
    print(':)')

filter shape: (32, 3, 3, 3)
input shape: (1, 224, 224, 3)
stride_h: 2
stride_w: 2
dilate_h_factor: 1
dilate_w_factor: 1
padding: SAME [1, 0, 1, 0]
output shape: (1, 112, 112, 32)
:)
